In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'tiêu cực'
    elif rating == 3:
        return 'trung tính'
    elif rating in [4, 5]:
        return 'tích cực'
    else:
        return 'không rõ'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      Điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                      Bảo hành ít quá, chỉ 12 tháng    tiêu cực
3            5                             Sản phẩm mượt, chạy êm    tích cực
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])

Star Rating    0
Comment        0
Sentiment      0
dtype: int64


In [5]:
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt  tích cực
76              5                                    Rất ok  tích cực
107             2                             pin tụt nhanh  tiêu cực
135             5                              sản phẩm tốt  tích cực
173             2                             hao pin nhanh  tiêu cực
177             5                              sản phẩm tốt  tích cực
181             5                                    Rất ok  tích cực
183             5                              sản phẩm tốt  tích cực
192             5                               sản phẩm ok  tích cực
205             5                               máy dùng ok  tích cực
225             4                                       Tốt  tích cực
227             5                               sản phẩm ok  tích cực
237             5                         

In [6]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                       bảo hành ít quá chỉ 12 tháng    tiêu cực
3            5                              sản phẩm mượt chạy êm    tích cực
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [7]:
from underthesea import word_tokenize

# Function to tokenize and build vocab using underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

# Tokenize comments and build vocabulary
tokenized_comments = df['Comment'].apply(tokenize_and_build_vocab_vietnamese)
all_tokens = [token for sublist in tokenized_comments for token in sublist]
vocab = FreqDist(all_tokens)

# Build vocab_list from vocab
vocab_list = list(vocab.keys())

print("Tổng số từ duy nhất:", len(vocab_list))
print("Các từ và tần suất xuất hiện:")
for word, frequency in vocab.most_common(10):  # Print only top 10 words
    print(f"{word}: {frequency}")


Tổng số từ duy nhất: 2386
Các từ và tần suất xuất hiện:
máy: 715
không: 659
dùng: 593
mua: 472
được: 422
rất: 409
pin: 347
tốt: 338
mình: 322
thì: 319


In [8]:
# Extract the list of reviews X
X = df['Comment'].values.tolist()
# Extract the labels y
y = df['Sentiment'].values.tolist()

In [9]:
# Use TfidfVectorizer to convert each review in X to a vector of number
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)
vocabulary = vectorizer.get_feature_names_out()

In [10]:
# Split data into training and temporary (validation + test) sets
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Split temporary set into validation and final test sets
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)


In [11]:
# Initialize Naive Bayes and SVM classifiers
nb_classifier = MultinomialNB()
svm_classifier = SVC(kernel='linear', probability=True)

In [12]:
# Create an ensemble using VotingClassifier
ensemble_classifier = VotingClassifier(estimators=[
    ('nb', nb_classifier),
    ('svm', svm_classifier)
], voting='soft')  # Use 'soft' voting for probabilities


In [13]:
#Train the ensemble classifier
ensemble_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('svm', SVC(kernel='linear', probability=True))],
                 voting='soft')

In [14]:
# Evaluate ensemble classifier on test set
accuracy = ensemble_classifier.score(X_test, y_test)
print(f"Ensemble Classifier Test Accuracy: {accuracy}")


Ensemble Classifier Test Accuracy: 0.78


In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Đã huấn luyện mô hình ensemble_classifier từ trước

# Dự đoán trên tập test
y_pred = ensemble_classifier.predict(X_test)

# Tính toán precision, recall và F1-score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Precision: 0.76
Recall: 0.78
F1-score: 0.74
